![alt](pics/siec.PNG)

In [1]:
from utils import Settings
from openai import OpenAI
import json
import requests

api_key = Settings().openai_api_key
client = OpenAI(api_key=api_key)

**Dostępne endpointy:**
- `GET /api/users?page=[page_id]` - lista pracowników (paginacja)
- `GET /api/users/[user_id]` - informacje o konkretnym pracowniku
- `DELETE /api/users/[user_id]` - usunięcie pracownika

In [2]:
response_all_users = requests.get('https://reqres.in/api/users?page=2', headers={"x-api-key": "reqres-free-v1"})
response_single_user = requests.get('https://reqres.in/api/users/2', headers={"x-api-key": "reqres-free-v1"})

In [3]:
response_single_user.json()

{'data': {'id': 2,
  'email': 'janet.weaver@reqres.in',
  'first_name': 'Janet',
  'last_name': 'Weaver',
  'avatar': 'https://reqres.in/img/faces/2-image.jpg'},
 'support': {'url': 'https://contentcaddy.io?utm_source=reqres&utm_medium=json&utm_campaign=referral',
  'text': 'Tired of writing endless social media content? Let Content Caddy generate it for you.'},
 '_meta': {'powered_by': '🚀 ReqRes - Deploy backends in 30 seconds',
  'upgrade_url': 'https://app.reqres.in/upgrade',
  'docs_url': 'https://reqres.in',
  'template_gallery': 'https://app.reqres.in/templates',
  'message': 'This API is powered by ReqRes. Deploy your own backend in 30 seconds!',
  'features': ['30 Second Backend Templates',
   'Custom API Endpoints',
   'Data Persistence',
   'Real-time Analytics'],
  'upgrade_cta': 'Upgrade to Pro for unlimited requests, custom endpoints, and data persistence'}}

In [4]:
response_all_users.json()['data']

[{'id': 7,
  'email': 'michael.lawson@reqres.in',
  'first_name': 'Michael',
  'last_name': 'Lawson',
  'avatar': 'https://reqres.in/img/faces/7-image.jpg'},
 {'id': 8,
  'email': 'lindsay.ferguson@reqres.in',
  'first_name': 'Lindsay',
  'last_name': 'Ferguson',
  'avatar': 'https://reqres.in/img/faces/8-image.jpg'},
 {'id': 9,
  'email': 'tobias.funke@reqres.in',
  'first_name': 'Tobias',
  'last_name': 'Funke',
  'avatar': 'https://reqres.in/img/faces/9-image.jpg'},
 {'id': 10,
  'email': 'byron.fields@reqres.in',
  'first_name': 'Byron',
  'last_name': 'Fields',
  'avatar': 'https://reqres.in/img/faces/10-image.jpg'},
 {'id': 11,
  'email': 'george.edwards@reqres.in',
  'first_name': 'George',
  'last_name': 'Edwards',
  'avatar': 'https://reqres.in/img/faces/11-image.jpg'},
 {'id': 12,
  'email': 'rachel.howell@reqres.in',
  'first_name': 'Rachel',
  'last_name': 'Howell',
  'avatar': 'https://reqres.in/img/faces/12-image.jpg'}]

In [5]:
response_delete = requests.delete('https://reqres.in/api/users/12', headers={"x-api-key": "reqres-free-v1"})

In [6]:
response_delete.status_code

204

In [7]:
def call_rest_api(http_method, endpoint_url):
  url = f"https://reqres.in/{endpoint_url}"
  response = None
  if http_method == "GET":
    response = requests.get(url, headers={"x-api-key": "reqres-free-v1"})
  elif http_method == "DELETE":
    response = requests.delete(url, headers={"x-api-key": "reqres-free-v1"})
  else:
    raise ValueError(http_method)

  if response.status_code == 200:
    return json.dumps(response.json())
  else:
    return f"Status code: {response.status_code}"

In [8]:
call_rest_api("GET", 'api/users/1')

'{"data": {"id": 1, "email": "george.bluth@reqres.in", "first_name": "George", "last_name": "Bluth", "avatar": "https://reqres.in/img/faces/1-image.jpg"}, "support": {"url": "https://contentcaddy.io?utm_source=reqres&utm_medium=json&utm_campaign=referral", "text": "Tired of writing endless social media content? Let Content Caddy generate it for you."}, "_meta": {"powered_by": "\\ud83d\\ude80 ReqRes - Deploy backends in 30 seconds", "upgrade_url": "https://app.reqres.in/upgrade", "docs_url": "https://reqres.in", "template_gallery": "https://app.reqres.in/templates", "message": "This API is powered by ReqRes. Deploy your own backend in 30 seconds!", "features": ["30 Second Backend Templates", "Custom API Endpoints", "Data Persistence", "Real-time Analytics"], "upgrade_cta": "Upgrade to Pro for unlimited requests, custom endpoints, and data persistence"}}'

In [9]:
call_rest_api("DELETE", 'api/users/12')

'Status code: 204'

In [10]:
functions = [
    {
        "type": "function",
        "function": {
            "name": "call_rest_api",
            "description": "Sends a request to the REST API",
            "parameters": {
                "type": "object",
                "properties": {
                    "http_method": {
                        "type": "string",
                        "description": "The HTTP method to be used",
                        "enum": ["GET", "DELETE"]
                    },
                    "endpoint_url": {
                        "type": "string",
                        "description": "The URL of the endpoint. For example: api/users/2 or api/users?page=1",
                    }
                },
                "required": ["http_method", "endpoint_url"]
            }
        }
    }
]

In [11]:
available_apis = [
        {'method': 'GET', 'url': '/api/users?page=[page_id]', 'description': 'Lists employees. The response is paginated. You may need to request more than one to get them all. For example,/api/users?page=2.'},
        {'method': 'GET', 'url': '/api/users/[user_id]', 'description': 'Returns information about the employee identified by the given id. For example,/api/users/2'},
        {'method': 'DELETE', 'url': '/api/users/[user_id]', 'description': 'Removes the employee identified by the given id. for example when you are asked to fire someone. Before you call this function, find the employee information and make sure the id is correct. Do NOT call this function if you didn\'t retrieve user info. Iterate over all pages until you find it or make sure it doesn\'t exist'}
    ]

In [12]:
system_messages = [
    {"role": "system", "content": "You are an HR helper who makes API calls on behalf of an HR representative"},
    {"role": "system", "content": "You have access to the following APIs: " + json.dumps(available_apis)},
    {"role": "system", "content": "If a function requires an identifier, list all employees first to find the proper value. You may need to list more than one page"},
    {"role": "system", "content": "If you were asked to create, update, or delete a user, perform the action and reply with a confirmation telling what you have done."}
]

#### pętla wywołań agenta

In [ ]:
messages = system_messages.copy()

messages.append({"role": "user", "content": "Find all emails"})

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=functions
)

assistant_message = response.choices[0].message
messages.append(assistant_message)


if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print(function_args)

        if function_name == "call_rest_api":
            result = call_rest_api(**function_args)
        else:
            result = f"Nieznane narzędzie: {function_name}"

        tool_message = {
            "role": "tool",
            "name": function_name,
            "tool_call_id": tool_call.id, 
            "content": result
        }
        messages.append(tool_message)

    followup = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    final_message = followup.choices[0].message
    messages.append(final_message)

{'http_method': 'GET', 'endpoint_url': '/api/users?page=1'}
